table 1
- mission type
- GCS
- NACA
- age
- patient sex
- physician sex
- experience of physician
- age physician
- missions per physician
- difficult terrain
- circulaiton insufficient
- breathing insufficient
- duration of mission (min)
- night time mission
- trauma
- VAS at scene 
- VAS at hospital admission
- VAS reduction 
- administered analgesic
-- fenta / fenta dose
-- morphine / morphine dose
-- ketamine / ketamine dose

In [ ]:
import pandas as pd
import re
import numpy as np
from operator import itemgetter
from tableone import TableOne

In [ ]:
data_path = '/Users/jk1/Library/CloudStorage/OneDrive-unige.ch/icu_research/prehospital/analgesia/data/Masterarbeit Analgesie_24.07.2025.xlsx'
medic_data_path = '/Users/jk1/Library/CloudStorage/OneDrive-unige.ch/icu_research/prehospital/analgesia/data/Liste Notärzte-1.xlsx'

In [ ]:
data_df = pd.read_excel(data_path)
medic_df = pd.read_excel(medic_data_path)

In [ ]:
medic_df = medic_df.drop_duplicates('Mitglieder mit Einsatzfunktion')
medic_df.rename(columns={'Sex m/w': 'physician_sex'}, inplace=True)
data_df = data_df.merge(medic_df, how='left', left_on='Mitglieder mit Einsatzfunktion', right_on='Mitglieder mit Einsatzfunktion')

In [ ]:
duplicates = data_df[data_df["SNZ Ereignis Nr. "].duplicated()]["SNZ Ereignis Nr. "]
print(f'Duplicates found: {duplicates.nunique()}')
# drop duplicates
data_df = data_df.drop_duplicates(subset=["SNZ Ereignis Nr. "])

In [ ]:
n_vas_under4 = data_df[data_df["VAS_on_scene"] <= 3].shape[0]
print(f'Excluded {n_vas_under4} patients with VAS <= 3')

data_df = data_df[data_df["VAS_on_scene"] > 3]

In [ ]:
data_df.shape

In [ ]:
(data_df["Alter "] >= 16).sum(), (data_df["Alter "] < 16).sum()

In [ ]:
adult_df = data_df[data_df["Alter "] >= 16]
pediatric_df = data_df[data_df["Alter "] < 16]

In [ ]:
# print all columns names containing "GCS" in any of data_df.columns
print([col for col in data_df.columns if "Geschlecht" in col])

In [ ]:
column = "Alle Medikamente"
data_df[column].nunique(), data_df[column].unique(), data_df[column].isna().sum(), data_df[column].value_counts()

In [ ]:
def get_categorical_str(df, column_name, category, total):
    """
    Get categorical string features from the DataFrame.
    """
    count = df[df[column_name] == category].shape[0]
    return f'{count} ({count/total:.1%})'

def get_continuous_str(df, column_name, total):
    """
    Get continuous string features from the DataFrame.
    """
    median = df[column_name].median()
    q1 = df[column_name].quantile(0.25)
    q3 = df[column_name].quantile(0.75)
    return f'{median:.1f} [{q1:.1f} - {q3:.1f}]'

def get_multi_label_counts(data_df, multi_label_column):
    data_df[multi_label_column] = data_df[multi_label_column].replace(999, pd.NA)
    label_counter = {}
    # iterate through the rows
    for index, row in data_df.iterrows():
        # split by comma then strip spaces
        labels = [label.strip() for label in re.split('; |, ', str(row[multi_label_column]))]
        # if label not in the dict, add it
        for label in labels:
            if label == 'nan' or label == '<NA>':
                continue
            if label not in label_counter:
                label_counter[label] = 1
            else:
                label_counter[label] += 1

    # sort the dictionary by value
    sorted_label_counter = dict(sorted(label_counter.items(), key=lambda item: item[1], reverse=True))
    return sorted_label_counter

In [ ]:
def table1(df):
    """
    Create a summary table of the data.
    """
    pop_df = pd.DataFrame()
    str_df = pd.DataFrame()

    # get the number of patients
    n_patients = len(df)
    pop_df['n_patients'] = [n_patients]
    str_df['n_patients'] = [n_patients]

    # mission type (primary vs secondary)
    str_df['primary_mission'] = get_categorical_str(df, 'Einsatzart', 'Primär', n_patients)
    str_df['secondary_mission'] = get_categorical_str(df, 'Einsatzart', 'Sekundär', n_patients)

    # GCS (13, 14, 15)
    str_df['GCS 13'] = get_categorical_str(df, 'GCS', 13, n_patients)
    str_df['GCS 14'] = get_categorical_str(df, 'GCS', 14, n_patients)
    str_df['GCS 15'] = get_categorical_str(df, 'GCS', 15, n_patients)

    # - age
    str_df['age'] = get_continuous_str(df, 'Alter ', n_patients)

    # - patient sex
    str_df['patient_sex_male'] = get_categorical_str(df, 'Geschlecht', 'Männlich', n_patients)

    # - physician sex
    str_df['physician_sex_male'] = get_categorical_str(df, 'physician_sex', 'm', n_patients)
    # - experience of physician
    # - age physician
    # - missions per physician
    
    # - difficult terrain
    # extraction with whinch
    extraction_methods = get_multi_label_counts(df, "Bergungen")
    
    n_whinch_extractions = np.sum(itemgetter(*[k for k, v in extraction_methods.items() if 'Winde' in k])(extraction_methods))
    str_df['n_whinch_extractions'] = f'{n_whinch_extractions} ({n_whinch_extractions/n_patients:.1%})'

    # - circulaiton insufficient
    # - breathing insufficient
    # - duration of mission (min)
    # mission time in minutes
    df['mission_duration'] = (pd.to_datetime(df['Übergabezeit'], format='%d.%m.%Y %H:%M:%S') - pd.to_datetime(df['Erstbefund'], format='%d.%m.%Y %H:%M:%S')).dt.total_seconds() / 60
    str_df['mission_duration'] = get_continuous_str(df, 'mission_duration', n_patients)
    
    # - night time mission
    str_df['night_time_mission'] = get_categorical_str(df, 'Tag oder Nacht', 'Nacht', n_patients)

    # - trauma
    n_trauma = get_multi_label_counts(df, "Einteilung (reduziert)")['Unfall']
    str_df['trauma'] = f'{n_trauma} ({n_trauma/n_patients:.1%})'

    # - VAS at scene 
    str_df['VAS_on_scene'] = get_continuous_str(df, 'VAS_on_scene', n_patients)
    # - VAS at hospital admission
    str_df['VAS_at_hospital_admission'] = get_continuous_str(df, 'VAS_on_arrival', n_patients)
    # - VAS reduction
    df['VAS_reduction'] = df['VAS_on_scene'] - df['VAS_on_arrival']
    str_df['VAS_reduction'] = get_continuous_str(df, 'VAS_reduction', n_patients)

    # - administered analgesic
    # -- fenta / fenta dose
    # -- morphine / morphine dose
    # -- ketamine / ketamine dose
    df['fentanyl_dose'] = 0
    df['ketamine_dose'] = 0
    df['esketamine_dose'] = 0
    df['morphine_dose'] = 0
    df['Alle Medikamente'] = df['Alle Medikamente'].str.replace(',', ';')  # replace commas with semicolons for consistency
    for i, row in df.iterrows():
        if pd.isna(row['Alle Medikamente']) or row['Alle Medikamente'] == 0:
            continue
        for analgetic in row['Alle Medikamente'].split(';'):
            if analgetic.strip() == '':
                continue
            # remove mcg or mg from dose
            if '7IE' in analgetic:
                    print(f"Skipping dose with 7IE: {analgetic}")
                    continue

            analgetic = analgetic.replace('mcg', '').replace('mg', '').strip()
            if 'Fentanyl' in analgetic and '/h' not in analgetic:
                dose = analgetic.split('Fentanyl')[-1].strip()
                df.at[i, 'fentanyl_dose'] += float(dose) 
            elif 'Fentanyl' in analgetic and '/h' in analgetic:
                dose = analgetic.split('Fentanyl')[-1].strip().replace('/h', '')
                dose = float(dose) * df.at[i, 'mission_duration']  
                df.at[i, 'fentanyl_dose'] += float(dose)
            elif 'Ketamin' in analgetic or 'Ketamine' in analgetic:
                dose = analgetic.split('Ketamin')[-1].strip()
                df.at[i, 'ketamine_dose'] += float(dose)
            elif 'Esketamin' in analgetic:
                dose = analgetic.split('Esketamin')[-1].strip()
                df.at[i, 'esketamine_dose'] += float(dose)
            elif 'Morphin' in analgetic or 'Morphine' in analgetic:
                dose = analgetic.split('Morphin')[-1].strip()
                df.at[i, 'morphine_dose'] += float(dose)

    # fentanyl given
    df['fentanyl_given'] = df['fentanyl_dose'] > 0
    # str_df['fentanyl'] = get_categorical_str(df, 'fentanyl_given', True, n_patients)
    # str_df['fentanyl_dose'] = get_continuous_str(df, 'fentanyl_dose', n_patients)

    # morphine given
    df['morphine_given'] = df['morphine_dose'] > 0
    # str_df['morphine'] = get_categorical_str(df, 'morphine_given', True, n_patients)
    # str_df['morphine_dose'] = get_continuous_str(df, 'morphine_dose', n_patients)

    # ketamine given
    df['ketamine_given'] = df['ketamine_dose'] > 0
    # str_df['ketamine'] = get_categorical_str(df, 'ketamine_given', True, n_patients)
    # str_df['ketamine_dose'] = get_continuous_str(df, 'ketamine_dose', n_patients)

    # esketamine given
    df['esketamine_given'] = df['esketamine_dose'] > 0
    # str_df['esketamine'] = get_categorical_str(df, 'esketamine_given', True, n_patients)
    # str_df['esketamine_dose'] = get_continuous_str(df, 'esketamine_dose', n_patients)

    df['any_ketamine_given'] = df['ketamine_given'] | df['esketamine_given']
    df['any_ketamine_dose'] = df['ketamine_dose'] + df['esketamine_dose']
    str_df['any_ketamine'] = get_categorical_str(df, 'any_ketamine_given', True, n_patients)
    str_df['any_ketamine_dose'] = get_continuous_str(df, 'any_ketamine_dose', n_patients)

    df['any_opioid_given'] = df['fentanyl_given'] | df['morphine_given']
    df['any_opioid_dose'] = df['fentanyl_dose'] + df['morphine_dose']
    str_df['any_opioid'] = get_categorical_str(df, 'any_opioid_given', True, n_patients)
    str_df['any_opioid_dose'] = get_continuous_str(df, 'any_opioid_dose', n_patients)

    return str_df.T

In [ ]:
insufficient_adult_df = adult_df[adult_df['VAS_on_arrival'] > 3]
sufficient_adult_df = adult_df[adult_df['VAS_on_arrival'] <= 3]
insufficient_pediatric_df = pediatric_df[pediatric_df['VAS_on_arrival'] > 3]
sufficient_pediatric_df = pediatric_df[pediatric_df['VAS_on_arrival'] <= 3]

In [ ]:
adult_table1 = table1(adult_df)
sufficient_adult_df = table1(sufficient_adult_df)
insufficient_adult_table1 = table1(insufficient_adult_df)

pediatric_table1 = table1(pediatric_df)
sufficient_pediatric_table1 = table1(sufficient_pediatric_df)
insufficient_pediatric_table1 = table1(insufficient_pediatric_df)


In [ ]:
overall_adult_table1 = pd.concat([adult_table1, sufficient_adult_df, insufficient_adult_table1], axis=1, keys=['Adult', 'Sufficient Adult', 'Insufficient Adult'])
overall_pediatric_table1 = pd.concat([pediatric_table1, sufficient_pediatric_table1, insufficient_pediatric_table1], axis=1, keys=['Pediatric', 'Sufficient Pediatric', 'Insufficient Pediatric'])

In [ ]:
# overall_adult_table1.to_csv('/Users/jk1/Library/CloudStorage/OneDrive-unige.ch/icu_research/prehospital/analgesia/adult_table1.csv')
# overall_pediatric_table1.to_csv('/Users/jk1/Library/CloudStorage/OneDrive-unige.ch/icu_research/prehospital/analgesia/pediatric_table1.csv')

In [ ]:
# Prepare data for tableone analysis
adult_df_copy = adult_df.copy()

# Create analgesia adequacy grouping variable
adult_df_copy['analgesia_adequacy'] = adult_df_copy['VAS_on_arrival'].apply(
    lambda x: 'Sufficient' if x <= 3 else 'Insufficient'
)

# Calculate mission duration
adult_df_copy['mission_duration'] = (
    pd.to_datetime(adult_df_copy['Übergabezeit'], format='%d.%m.%Y %H:%M:%S') - 
    pd.to_datetime(adult_df_copy['Erstbefund'], format='%d.%m.%Y %H:%M:%S')
).dt.total_seconds() / 60

# Calculate VAS reduction
adult_df_copy['VAS_reduction'] = adult_df_copy['VAS_on_scene'] - adult_df_copy['VAS_on_arrival']

# Process medication data
adult_df_copy['fentanyl_dose'] = 0
adult_df_copy['ketamine_dose'] = 0
adult_df_copy['esketamine_dose'] = 0
adult_df_copy['morphine_dose'] = 0
adult_df_copy['Alle Medikamente'] = adult_df_copy['Alle Medikamente'].str.replace(',', ';')

for i, row in adult_df_copy.iterrows():
    if pd.isna(row['Alle Medikamente']) or row['Alle Medikamente'] == 0:
        continue
    for analgetic in row['Alle Medikamente'].split(';'):
        if analgetic.strip() == '':
            continue
        if '7IE' in analgetic:
            continue
        analgetic = analgetic.replace('mcg', '').replace('mg', '').strip()
        try:
            if 'Fentanyl' in analgetic and '/h' not in analgetic:
                dose = analgetic.split('Fentanyl')[-1].strip()
                adult_df_copy.at[i, 'fentanyl_dose'] += float(dose) 
            elif 'Fentanyl' in analgetic and '/h' in analgetic:
                dose = analgetic.split('Fentanyl')[-1].strip().replace('/h', '')
                dose = float(dose) * adult_df_copy.at[i, 'mission_duration']  
                adult_df_copy.at[i, 'fentanyl_dose'] += float(dose)
            elif 'Ketamin' in analgetic:
                dose = analgetic.split('Ketamin')[-1].strip()
                adult_df_copy.at[i, 'ketamine_dose'] += float(dose)
            elif 'Esketamin' in analgetic:
                dose = analgetic.split('Esketamin')[-1].strip()
                adult_df_copy.at[i, 'esketamine_dose'] += float(dose)
            elif 'Morphine' in analgetic:
                dose = analgetic.split('Morphine')[-1].strip()
                adult_df_copy.at[i, 'morphine_dose'] += float(dose)
        except ValueError:
            # Skip medications with non-numeric doses
            continue

# Create binary medication variables
adult_df_copy['fentanyl_given'] = adult_df_copy['fentanyl_dose'] > 0
adult_df_copy['morphine_given'] = adult_df_copy['morphine_dose'] > 0
adult_df_copy['ketamine_given'] = adult_df_copy['ketamine_dose'] > 0
adult_df_copy['esketamine_given'] = adult_df_copy['esketamine_dose'] > 0

# Combine ketamine and esketamine into one category
adult_df_copy['any_ketamine_dose'] = adult_df_copy['ketamine_dose'] + adult_df_copy['esketamine_dose']
adult_df_copy['any_ketamine_given'] = (adult_df_copy['ketamine_dose'] > 0) | (adult_df_copy['esketamine_dose'] > 0)

# Combine morphine and fentanyl into one opiate category
adult_df_copy['any_opiate_dose'] = adult_df_copy['morphine_dose'] + adult_df_copy['fentanyl_dose']
adult_df_copy['any_opiate_given'] = (adult_df_copy['morphine_dose'] > 0) | (adult_df_copy['fentanyl_dose'] > 0)

# Create trauma variable
adult_df_copy['trauma'] = adult_df_copy['Einteilung (reduziert)'].str.contains('Unfall', na=False)

# Create night mission variable
adult_df_copy['night_mission'] = adult_df_copy['Tag oder Nacht'] == 'Nacht'

# Create winch extraction variable
adult_df_copy['winch_extraction'] = adult_df_copy['Bergungen'].str.contains('Winde', na=False)

# Create male patient variable
adult_df_copy['male_patient'] = adult_df_copy['Geschlecht'] == 'Männlich'

# Create male physician variable  
adult_df_copy['male_physician'] = adult_df_copy['physician_sex'] == 'm'

# Create primary mission variable
adult_df_copy['primary_mission'] = adult_df_copy['Einsatzart'] == 'Primär'

print(f"Data prepared for {len(adult_df_copy)} adult patients")
print(f"Sufficient analgesia: {(adult_df_copy['analgesia_adequacy'] == 'Sufficient').sum()}")
print(f"Insufficient analgesia: {(adult_df_copy['analgesia_adequacy'] == 'Insufficient').sum()}")
adult_df_copy['male_physician'] = adult_df_copy['physician_sex'] == 'm'

# Create primary mission variable
adult_df_copy['primary_mission'] = adult_df_copy['Einsatzart'] == 'Primär'

In [ ]:
# Define variables for TableOne - only include variables with variation
columns = [
    'primary_mission',
    'GCS', 
    'Alter ',
    'male_patient',
    'male_physician',
    'winch_extraction',
    'night_mission',
    'trauma',
    'VAS_on_scene',
    'VAS_on_arrival', 
    'VAS_reduction',
    'any_opiate_given',  # Combined morphine and fentanyl
    'any_ketamine_given'  # Combined ketamine and esketamine
]

# Define categorical variables
categorical = [
    'primary_mission',
    'GCS',
    'male_patient', 
    'male_physician',
    'winch_extraction',
    'night_mission',
    'trauma',
    'any_opiate_given',  # Combined morphine and fentanyl
    'any_ketamine_given'  # Combined ketamine and esketamine
]

# Define non-normal variables (will be reported as median [IQR])
nonnormal = [
    'Alter ',
    'VAS_on_scene',
    'VAS_on_arrival',
    'VAS_reduction'
]

# Check for variables with variation before creating table
print("Checking variable distributions:")
for col in columns:
    if col in adult_df_copy.columns:
        unique_vals = adult_df_copy[col].nunique()
        print(f"{col}: {unique_vals} unique values")
    
# Remove variables with no variation
columns_with_variation = []
for col in columns:
    if col in adult_df_copy.columns and adult_df_copy[col].nunique() > 1:
        columns_with_variation.append(col)
        
print(f"\nIncluding {len(columns_with_variation)} variables with variation")

# Update categorical and nonnormal lists to only include variables with variation
print(f"Variables included: {columns_with_variation}")

# Update categorical and nonnormal lists to only include variables with variation
categorical_filtered = [c for c in categorical if c in columns_with_variation]
nonnormal_filtered = [c for c in nonnormal if c in columns_with_variation]

print(f"Categorical variables: {categorical_filtered}")
print(f"Non-normal variables: {nonnormal_filtered}")

# Ensure boolean variables are properly typed as integers for tableone
for col in categorical_filtered:
    if col in adult_df_copy.columns and adult_df_copy[col].dtype == 'bool':
        adult_df_copy[col] = adult_df_copy[col].astype(int)

# Create TableOne
table1_adult = TableOne(
    adult_df_copy, 
    columns=columns_with_variation,
    categorical=categorical_filtered,    nonnormal=nonnormal_filtered,    groupby='analgesia_adequacy',
    pval=True,
    label_suffix=True
)

print("\nAdult Patient Characteristics by Analgesia Adequacy")

print("\nAdult Patient Characteristics by Analgesia Adequacy")
print("=" * 60)
print(table1_adult.tabulate(tablefmt="grid"))

In [ ]:
# Create a more readable version and save to CSV
table1_df = table1_adult.tableone

# Display summary statistics
print("Summary of Table 1 with P-values:")
print("=" * 50)
print(f"Total patients: {len(adult_df_copy)}")
print(f"Sufficient analgesia: {(adult_df_copy['analgesia_adequacy'] == 'Sufficient').sum()}")
print(f"Insufficient analgesia: {(adult_df_copy['analgesia_adequacy'] == 'Insufficient').sum()}")

# Show variables with significant p-values (< 0.05)
if 'P-Value' in table1_df.columns:
    significant_vars = table1_df[table1_df['P-Value'] < 0.05]
    if len(significant_vars) > 0:
        print(f"\nVariables with significant differences (p < 0.05):")
        for var in significant_vars.index:
            p_val = significant_vars.loc[var, 'P-Value']
            print(f"  {var}: p = {p_val:.3f}")
    else:
        print("\nNo variables with significant differences (p < 0.05) found")

# Save table to CSV (uncomment to save)
# table1_df.to_csv('adult_table1_with_pvalues.csv')
print("\nTable created successfully with p-values!")
print("Use table1_adult.tableone to access the full DataFrame")
print("Use table1_adult.tabulate(tablefmt='grid') to display formatted table")

In [ ]:
# Prepare data for pediatric tableone analysis
pediatric_df_copy = pediatric_df.copy()

# Create analgesia adequacy grouping variable
pediatric_df_copy['analgesia_adequacy'] = pediatric_df_copy['VAS_on_arrival'].apply(
    lambda x: 'Sufficient' if x <= 3 else 'Insufficient'
)

# Calculate mission duration
pediatric_df_copy['mission_duration'] = (
    pd.to_datetime(pediatric_df_copy['Übergabezeit'], format='%d.%m.%Y %H:%M:%S') - 
    pd.to_datetime(pediatric_df_copy['Erstbefund'], format='%d.%m.%Y %H:%M:%S')
).dt.total_seconds() / 60

# Calculate VAS reduction
pediatric_df_copy['VAS_reduction'] = pediatric_df_copy['VAS_on_scene'] - pediatric_df_copy['VAS_on_arrival']

# Process medication data
pediatric_df_copy['fentanyl_dose'] = 0
pediatric_df_copy['ketamine_dose'] = 0
pediatric_df_copy['esketamine_dose'] = 0
pediatric_df_copy['morphine_dose'] = 0
pediatric_df_copy['Alle Medikamente'] = pediatric_df_copy['Alle Medikamente'].str.replace(',', ';')

for i, row in pediatric_df_copy.iterrows():
    if pd.isna(row['Alle Medikamente']) or row['Alle Medikamente'] == 0:
        continue
    for analgetic in row['Alle Medikamente'].split(';'):
        if analgetic.strip() == '':
            continue
        if '7IE' in analgetic:
            continue
        analgetic = analgetic.replace('mcg', '').replace('mg', '').strip()
        try:
            if 'Fentanyl' in analgetic and '/h' not in analgetic:
                dose = analgetic.split('Fentanyl')[-1].strip()
                pediatric_df_copy.at[i, 'fentanyl_dose'] += float(dose) 
            elif 'Fentanyl' in analgetic and '/h' in analgetic:
                dose = analgetic.split('Fentanyl')[-1].strip().replace('/h', '')
                dose = float(dose) * pediatric_df_copy.at[i, 'mission_duration']  
                pediatric_df_copy.at[i, 'fentanyl_dose'] += float(dose)
            elif 'Ketamin' in analgetic:
                dose = analgetic.split('Ketamin')[-1].strip()
                pediatric_df_copy.at[i, 'ketamine_dose'] += float(dose)
            elif 'Esketamin' in analgetic:
                dose = analgetic.split('Esketamin')[-1].strip()
                pediatric_df_copy.at[i, 'esketamine_dose'] += float(dose)
            elif 'Morphine' in analgetic:
                dose = analgetic.split('Morphine')[-1].strip()
                pediatric_df_copy.at[i, 'morphine_dose'] += float(dose)
        except ValueError:
            # Skip medications with non-numeric doses
            continue

# Create binary medication variables
pediatric_df_copy['fentanyl_given'] = pediatric_df_copy['fentanyl_dose'] > 0
pediatric_df_copy['morphine_given'] = pediatric_df_copy['morphine_dose'] > 0
pediatric_df_copy['ketamine_given'] = pediatric_df_copy['ketamine_dose'] > 0
pediatric_df_copy['esketamine_given'] = pediatric_df_copy['esketamine_dose'] > 0

# Combine ketamine and esketamine into one category
pediatric_df_copy['any_ketamine_dose'] = pediatric_df_copy['ketamine_dose'] + pediatric_df_copy['esketamine_dose']
pediatric_df_copy['any_ketamine_given'] = (pediatric_df_copy['ketamine_dose'] > 0) | (pediatric_df_copy['esketamine_dose'] > 0)

# Combine morphine and fentanyl into one opiate category
pediatric_df_copy['any_opiate_dose'] = pediatric_df_copy['morphine_dose'] + pediatric_df_copy['fentanyl_dose']
pediatric_df_copy['any_opiate_given'] = (pediatric_df_copy['morphine_dose'] > 0) | (pediatric_df_copy['fentanyl_dose'] > 0)

# Create trauma variable
pediatric_df_copy['trauma'] = pediatric_df_copy['Einteilung (reduziert)'].str.contains('Unfall', na=False)

# Create night mission variable
pediatric_df_copy['night_mission'] = pediatric_df_copy['Tag oder Nacht'] == 'Nacht'

# Create winch extraction variable
pediatric_df_copy['winch_extraction'] = pediatric_df_copy['Bergungen'].str.contains('Winde', na=False)

# Create male patient variable
pediatric_df_copy['male_patient'] = pediatric_df_copy['Geschlecht'] == 'Männlich'

# Create male physician variable  
pediatric_df_copy['male_physician'] = pediatric_df_copy['physician_sex'] == 'm'

# Create primary mission variable
pediatric_df_copy['primary_mission'] = pediatric_df_copy['Einsatzart'] == 'Primär'

print(f"Data prepared for {len(pediatric_df_copy)} pediatric patients")
print(f"Sufficient analgesia: {(pediatric_df_copy['analgesia_adequacy'] == 'Sufficient').sum()}")
print(f"Insufficient analgesia: {(pediatric_df_copy['analgesia_adequacy'] == 'Insufficient').sum()}")

In [ ]:
# Define variables for Pediatric TableOne - same as adult
pediatric_columns = [
    'primary_mission',
    'GCS', 
    'Alter ',
    'male_patient',
    'male_physician',
    'winch_extraction',
    'night_mission',
    'trauma',
    'VAS_on_scene',
    'VAS_on_arrival', 
    'VAS_reduction',
    'any_opiate_given',  # Combined morphine and fentanyl
    'any_ketamine_given'  # Combined ketamine and esketamine
]

# Define categorical variables
pediatric_categorical = [
    'primary_mission',
    'GCS',
    'male_patient', 
    'male_physician',
    'winch_extraction',
    'night_mission',
    'trauma',
    'any_opiate_given',  # Combined morphine and fentanyl
    'any_ketamine_given'  # Combined ketamine and esketamine
]

# Define non-normal variables (will be reported as median [IQR])
pediatric_nonnormal = [
    'Alter ',
    'VAS_on_scene',
    'VAS_on_arrival',
    'VAS_reduction'
]

# Check for variables with variation before creating table
print("Checking pediatric variable distributions:")
for col in pediatric_columns:
    if col in pediatric_df_copy.columns:
        unique_vals = pediatric_df_copy[col].nunique()
        print(f"{col}: {unique_vals} unique values")
    
# Remove variables with no variation
pediatric_columns_with_variation = []
for col in pediatric_columns:
    if col in pediatric_df_copy.columns and pediatric_df_copy[col].nunique() > 1:
        pediatric_columns_with_variation.append(col)
        
print(f"\nIncluding {len(pediatric_columns_with_variation)} pediatric variables with variation")
print(f"Variables included: {pediatric_columns_with_variation}")

# Update categorical and nonnormal lists to only include variables with variation
pediatric_categorical_filtered = [c for c in pediatric_categorical if c in pediatric_columns_with_variation]
pediatric_nonnormal_filtered = [c for c in pediatric_nonnormal if c in pediatric_columns_with_variation]

print(f"Categorical variables: {pediatric_categorical_filtered}")
print(f"Non-normal variables: {pediatric_nonnormal_filtered}")

# Ensure boolean variables are properly typed as integers for tableone
for col in pediatric_categorical_filtered:
    if col in pediatric_df_copy.columns and pediatric_df_copy[col].dtype == 'bool':
        pediatric_df_copy[col] = pediatric_df_copy[col].astype(int)

# Create Pediatric TableOne
table1_pediatric = TableOne(
    pediatric_df_copy, 
    columns=pediatric_columns_with_variation,
    categorical=pediatric_categorical_filtered,
    nonnormal=pediatric_nonnormal_filtered,
    groupby='analgesia_adequacy',
    pval=True,
    label_suffix=True
)

print("\nPediatric Patient Characteristics by Analgesia Adequacy")
print("=" * 60)
print(table1_pediatric.tabulate(tablefmt="grid"))

In [ ]:
# Create a summary for pediatric analysis
table1_pediatric_df = table1_pediatric.tableone

# Display summary statistics for pediatric patients
print("Summary of Pediatric Table 1 with P-values:")
print("=" * 50)
print(f"Total pediatric patients: {len(pediatric_df_copy)}")
print(f"Sufficient analgesia: {(pediatric_df_copy['analgesia_adequacy'] == 'Sufficient').sum()}")
print(f"Insufficient analgesia: {(pediatric_df_copy['analgesia_adequacy'] == 'Insufficient').sum()}")

# Show variables with significant p-values (< 0.05)
if 'P-Value' in table1_pediatric_df.columns:
    significant_vars_pediatric = table1_pediatric_df[table1_pediatric_df['P-Value'] < 0.05]
    if len(significant_vars_pediatric) > 0:
        print(f"\nPediatric variables with significant differences (p < 0.05):")
        for var in significant_vars_pediatric.index:
            p_val = significant_vars_pediatric.loc[var, 'P-Value']
            print(f"  {var}: p = {p_val:.3f}")
    else:
        print("\nNo pediatric variables with significant differences (p < 0.05) found")

# Save table to CSV (uncomment to save)
# table1_pediatric_df.to_csv('pediatric_table1_with_pvalues.csv')
print("\nPediatric table created successfully with p-values!")
print("Use table1_pediatric.tableone to access the full DataFrame")
print("Use table1_pediatric.tabulate(tablefmt='grid') to display formatted table")

In [ ]:
# table1_pediatric_df.to_csv('/Users/jk1/Library/CloudStorage/OneDrive-unige.ch/icu_research/prehospital/analgesia/pediatric_table1_with_pvalues.csv')
# table1_adult.to_csv('/Users/jk1/Library/CloudStorage/OneDrive-unige.ch/icu_research/prehospital/analgesia/adult_table1_with_pvalues.csv')